In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import os, errno
import glob

In [2]:
centroid_tracks = r"E:\Downloads\ManducaMultiSenseData\Step1\\"
visitInfo = r"E:\Downloads\ManducaMultiSenseData\v3\Step5_v3\\"

In [3]:
outpath = r"E:\Downloads\ManducaMultiSenseData\v3\Centroids_Relevant_v3\\"

In [4]:
mothVisit_list = glob.glob(visitInfo + "*.csv")
centroids = glob.glob(centroid_tracks + "*.csv")
flower_radius = pd.read_csv('E:\Downloads\ManducaMultiSenseData\\v3\Centroids_v3\FlowerImage\circle_params.csv')

In [5]:
df = pd.read_csv(mothVisit_list[0], dtype = 'float')
df.head()

,Unnamed: 0,In_Frame,Out_Frame,ProboscisDetect,DiscoveryTime
0,0.0,20377.0,20713.0,NaN,NaN
1,1.0,39912.0,41054.0,NaN,NaN
2,2.0,41723.0,41890.0,NaN,NaN
3,3.0,67784.0,69028.0,NaN,NaN
4,4.0,103803.0,104034.0,NaN,NaN


In [6]:
visit_num = [0, -1]

In [7]:
pix_thresh = 500
for file in mothVisit_list:
    df = pd.read_csv(file)
    
    a,b,c,_ = os.path.basename(file).split('_')
    name = a + '_' + b + '_' + c
    
    string = name + '.csv'
    matched_file = [c for c in centroids if string in c][0]
    cent_file = pd.read_csv(matched_file)
    cent_filtered_x_list, cent_filtered_y_list = [], []
    
    for i in cent_file.index.values:
        if cent_file.NumPixel.values[i] > pix_thresh:
            cent_filtered_x_list.append(cent_file.Centroid_hull_x.values[i])
            cent_filtered_y_list.append(cent_file.Centroid_hull_y.values[i])
        else:
            cent_filtered_x_list.append(np.nan)
            cent_filtered_y_list.append(np.nan)
            
    cent_file['centroid_filtered_x'] = cent_filtered_x_list
    cent_file['centroid_filtered_y'] = cent_filtered_y_list
   
    for v in visit_num:
        row = df.iloc[v,:]
        strt = int(row['In_Frame'])
        end = row['ProboscisDetect']
        
        if np.isnan(np.asarray(end)):
            end = row['Out_Frame']
        end = int(end)
        
        # get the radius
        r = flower_radius.loc[flower_radius.name == name, 'circ_radii'].values
        
        frameNums = cent_file[strt:end].index.values
        x = cent_file[strt:end].centroid_filtered_x.values*2.5/r
        y = cent_file[strt:end].centroid_filtered_y.values*2.5/r
        
        new_df = pd.DataFrame({"FrameNumber": frameNums, "X_position": x, "Y_position": y})
        if v == -1:
            visitname = 'last'
        else:
            visitname = 'first'
        new_df.to_csv(outpath + "\\"+ name + "_" + visitname + 'Visit' + ".csv")